In [ ]:
# IMPORT LIBRARIES & LOAD EXCEL
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

#  Check  file exists
file_name = '/content/project_data.xls'


if os.path.exists(file_name):
    print(f"Success! Found '{file_name}'. Loading data...")

    # 2. Read the Excel file

    df = pd.read_excel(file_name)

    #  first 5 rows
    print("Here is a preview of your data:")
    display(df.head())

else:
    print(f" Error: Could not find '{file_name}'.")


In [ ]:
# DATA  CLEANING

# Define the columns  months
month_cols = ['Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# Function to fix messy numbers
def clean_currency(x):
    if isinstance(x, str):
        # If it's text like "23%", remove % and convert to number
        return float(x.replace('%', ''))
    elif isinstance(x, (int, float)):
        # If Excel saved it as 0.23, convert to 23.0
        # If it's already 23.0, keep it as is.
        if x <= 1:
            return x * 100
        return x
    return 0

# Apply the cleaning to all month columns
for col in month_cols:
    df[col] = df[col].apply(clean_currency)

print("Data Cleaned! All percentages are now numbers.")
print(df[month_cols].head())

In [ ]:
# ANALYSIS & CLUSTERING
#This block calculates the Average, Volatility (Risk), and grouping similar stores.
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

#  Create Statistics
df['Average_Perf'] = df[month_cols].mean(axis=1) # Average
df['Volatility'] = df[month_cols].std(axis=1)    # Risk (Standard Deviation)
df['Momentum'] = df['Dec'] - df['Aug']           # Growth (Dec vs Aug)

#  Machine Learning (Clustering)
# group stores based on their stats
scaler = StandardScaler()
features = df[['Average_Perf', 'Volatility', 'Momentum']]
scaled_features = scaler.fit_transform(features)

kmeans = KMeans(n_clusters=4, random_state=42)
df['Cluster'] = kmeans.fit_predict(scaled_features)

# Name the clusters automatically
def name_clusters(df):
    summary = df.groupby('Cluster')['Average_Perf'].mean().sort_values(ascending=False)
    rank = summary.index.tolist()
    mapping = {
        rank[0]: 'High Performers',
        rank[1]: 'Growth/Volatile',
        rank[2]: 'Consistent Average',
        rank[3]: 'Low/Inactive'
    }
    return df['Cluster'].map(mapping)

df['Segment'] = name_clusters(df)

print(" Analysis Complete. Created Clusters.")
print("\n TOP 3 GROWING STORES ")
print(df.sort_values('Momentum', ascending=False)[['Store_Name', 'Momentum']].head(3))

print("\n--- TOP 3 CRASHING STORES ---")
print(df.sort_values('Momentum', ascending=True)[['Store_Name', 'Momentum']].head(3))

In [ ]:
#  INTERACTIVE PLOTS
import plotly.express as px
import plotly.graph_objects as go

print(" Plotly loaded Ready to create interactive charts.")

In [ ]:
#  RISK VS REWARD SCATTER

fig1 = px.scatter(
    df,
    x='Volatility',
    y='Average_Perf',
    color='Segment',
    hover_name='Store_Name',
    hover_data=['Branch', 'Momentum'],
    size='Average_Perf',             # Bigger dots = High Performer
    title='<b>Strategic Map: Risk vs Reward</b><br>(Hover over dots to see Store Names)',
    template='plotly_white',
    height=600
)

# Adding  vertical line to show  "High Risk" starts
fig1.add_vline(x=15, line_width=1, line_dash="dash", line_color="red")
fig1.add_annotation(x=16, y=90, text="High Risk Zone >>", showarrow=False)

fig1.show()

In [ ]:
# REGIONAL PERFORMANCE

# Sort data so the best region is on top
df_sorted = df.sort_values('Average_Perf', ascending=False)

fig2 = px.box(
    df_sorted,
    x='Branch',
    y='Average_Perf',
    color='Branch',
    points="all", # Showing store dots next to the box
    hover_name='Store_Name',
    title='<b>Regional Performance Comparison</b><br>(Which region is winning?)',
    template='plotly_white',
    height=500
)

fig2.update_layout(showlegend=False)
fig2.show()

In [ ]:
# PUNE HEATMAP

# 1. Filter for Pune only
pune_df = df[df['Branch'] == 'Pune'].set_index('Store_Name')[month_cols]
# we can add more branch like delhi to chech for the same

# 2. Create Heatmap
fig3 = px.imshow(
    pune_df,
    labels=dict(x="Month", y="Store", color="Performance %"),
    x=month_cols,
    y=pune_df.index,
    color_continuous_scale='Viridis', # Color scheme
    aspect="auto",
    title='<b>Deep Dive: Pune Region Volatility</b><br>(Yellow = High Perf, Purple = Low Perf)',
    height=700
)

fig3.show()

In [ ]:
# --- TREND LINES ---

#  Get Top 5 and Bottom 5 stores based on Momentum or  Growth
top_5 = df.nlargest(5, 'Momentum')
bottom_5 = df.nsmallest(5, 'Momentum')
combined_movers = pd.concat([top_5, bottom_5])

#  Melt data
melted_df = combined_movers.melt(
    id_vars=['Store_Name', 'Segment'],
    value_vars=month_cols,
    var_name='Month',
    value_name='Performance'
)

# Draw the Lines
fig4 = px.line(
    melted_df,
    x='Month',
    y='Performance',
    color='Store_Name',
    markers=True, # Add dots on the lines
    title='<b>Trend Analysis: Top 5 Growers vs. Top 5 Crashers</b><br>(Click legend to hide/show lines)',
    template='plotly_white',
    height=600
)

# Highlight the 100% mark
fig4.add_hline(y=100, line_dash="dot", annotation_text="Max Possible (100%)", annotation_position="top left")

fig4.show()

In [ ]:
#  RELIABILITY ANALYSIS

# 1. Calculate Coefficient of Variation (CV)
# Formula: Volatility / Average. Lower is better.
df['Reliability_Score'] = df['Volatility'] / df['Average_Perf']

# Filter out zeros to avoid errors
active_stores = df[df['Average_Perf'] > 5].copy()

# Sort: Lowest score is most reliable
most_reliable = active_stores.sort_values('Reliability_Score').head(10)

# Visualize
fig5 = px.bar(
    most_reliable,
    x='Reliability_Score',
    y='Store_Name',
    color='Average_Perf',
    orientation='h',
    title='<b>Top 10 Most Consistent Stores (The "Anchors")</b><br>(Short bars = Very Stable. Color = Performance)',
    labels={'Reliability_Score': 'Instability Score (Lower is Better)'},
    template='plotly_white',
    color_continuous_scale='Teal'
)

fig5.update_layout(yaxis=dict(autorange="reversed")) # Top store at top
fig5.show()

print("STRATEGY TIP: These stores are your 'Anchors'. Use them to pilot new products because their baseline is stable.")

In [ ]:
# --REGIONAL HEALTH CHECK (ZOMBIES or dead shop or low performer ) ---

# Tag stores as Zombie or Active
# anything averaging under 5% is effectively dead
df['Health_Status'] = np.where(df['Average_Perf'] < 5, 'Zombie (Inactive)', 'Healthy (Active)')

# 2. Count them by Region
regional_health = df.groupby(['Branch', 'Health_Status']).size().reset_index(name='Count')

# 3. Visualize
fig6 = px.bar(
    regional_health,
    x='Branch',
    y='Count',
    color='Health_Status',
    title='<b>Regional Health: Where are the "Zombie" Stores?</b><br>(Red bars = Dead Capital/Potential Closures)',
    color_discrete_map={'Zombie (Inactive)': '#EF553B', 'Healthy (Active)': '#00CC96'},
    template='plotly_white',
    barmode='stack'
)

fig6.show()

print("STRATEGY TIP: Look at 'Thane' and 'Mumbai'. The red bars represent rent paid for zero return. Initiate closures or re-staffing immediately.")

In [ ]:
#  AI FORECASTING & INTERACTIVE PREDICTOR
from sklearn.linear_model import LinearRegression
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.express as px
import plotly.graph_objects as go

# ---------------------------------------------------------
# STEP 1: TRAIN THE AI MODEL
# ---------------------------------------------------------
# X as Features = Aug, Sep, Oct, Nov
# y as Target   = Dec
X_train_data = df[['Aug', 'Sep', 'Oct', 'Nov']]
y_train_data = df['Dec']

model = LinearRegression()
model.fit(X_train_data, y_train_data)

# Predict January (using Sep, Oct, Nov, Dec as input)
X_future = df[['Sep', 'Oct', 'Nov', 'Dec']]
X_future.columns = ['Aug', 'Sep', 'Oct', 'Nov'] # Rename to match training columns
df['AI_Jan_Prediction'] = model.predict(X_future)
df['AI_Jan_Prediction'] = df['AI_Jan_Prediction'].clip(lower=0, upper=100)

print(" AI Model Trained(Linear Regression based ml Model) & January Predictions Generated.")

# ---------------------------------------------------------
#  THE FIXED PREDICTION FUNCTION
# ---------------------------------------------------------
def predict_store(store_name_input):


    store_match = df[df['Store_Name'].str.contains(store_name_input, case=False, na=False, regex=False)]

    if store_match.empty:

        # We turn regex=True back on for generic words, but escape special chars if needed
        try:
            store_match = df[df['Store_Name'].str.contains(store_name_input, case=False, na=False)]
        except:
            print(f" Could not find store: {store_name_input}")
            return

    if store_match.empty:
        print(f" Store '{store_name_input}' not found.")
        return

    # Pick the first result
    store_row = store_match.iloc[0]
    name = store_row['Store_Name']

    # Get Data
    history = [store_row['Aug'], store_row['Sep'], store_row['Oct'], store_row['Nov'], store_row['Dec']]
    prediction = store_row['AI_Jan_Prediction']
    months = ['Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'Jan (Pred)']
    values = history + [prediction]

    # VISUALIZE
    print(f"\n PREDICTION FOR: {name}")
    print(f"   - Current Trend: {'UP 📈' if prediction > history[-1] else 'DOWN 📉'}")
    print(f"   - AI Forecast for Jan: {prediction:.1f}%")

    fig = px.line(
        x=months,
        y=values,
        markers=True,
        title=f'<b>AI Forecast: {name}</b>',
        template='plotly_white'
    )

    # Add Red Dotted Line for the Prediction
    fig.add_trace(go.Scatter(
        x=['Dec', 'Jan (Pred)'],
        y=[history[-1], prediction],
        mode='lines+markers',
        line=dict(color='red', width=3, dash='dot'),
        name='AI Forecast'
    ))

    fig.update_layout(showlegend=False, yaxis_range=[0, 110])
    fig.show()

# ---------------------------------------------------------
# STEP 3: DROPDOWN MENU
# ---------------------------------------------------------
all_stores = sorted(df['Store_Name'].unique().tolist())

print("\n SELECT A STORE TO PREDICT:")

dropdown = widgets.Dropdown(
    options=all_stores,
    description='Store:',
    disabled=False,
)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        clear_output(wait=True)
        display(dropdown)
        predict_store(change['new'])

dropdown.observe(on_change)
display(dropdown)

# Initialize with the first store
predict_store(all_stores[0])